In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from IPython import display

try:
  import torch


except ModuleNotFoundError:
  %pip install -qq torch
  import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data

import collections
import re
import random
import requests
import hashlib
import time




np.random.seed(seed=1)
torch.manual_seed(seed=1)

!mkdir figures

**Data**

In [ ]:
class SeqDataLoader:
  """An interator to load sequence data."""

  def __init__(self, batch_size, num_steps, use_random_iter, max_tokens):
    if use_random_iter:
      self.data_iter_fn = seq_data_iter_random
    else:
      self.data_iter_fn = self.seq_data_iter_sequential

    self.corpus, self.vocab = load_corpus_time_machine(max_tokens)
    self.batch_size, self.num_steps = batch_size, num_steps

  def __iter__(self):
    return self.data_iter_fn(self.corpus, self.batch_size, self.num_steps)


class Vocab:
  """Vocabulary for text."""
  def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
    if tokens is None:
      tokens = []
    if reserved_tokens is None:
      reserved_tokens = []
    counter = count_corpus(tokens)



    # Sort according to frequencies




In [ ]:
def seq_data_iter_random():
  pass


def seq_data_iter_sequential():
  pass


def load_corpus_time_machine():
  pass



def count_corpus(tokens):
  pass